<a href="https://colab.research.google.com/github/Vlopes7/Transfer-Learning-/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!pip install -q tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_val), ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)

print(ds_info)

In [ ]:
IMG_SIZE = (160, 160)
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0
    return image, label

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # congela

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=3
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_imgs = 7

ds_val_shuffled = ds_val.shuffle(1000)

for images, labels in ds_val_shuffled.take(1):
    preds = model.predict(images)

    plt.figure(figsize=(15, 5))
    for i in range(num_imgs):
        img = images[i].numpy()
        prob_dog = preds[i][0]

        plt.subplot(1, num_imgs, i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"DOG: {prob_dog:.2f}\nCAT: {1-prob_dog:.2f}")

    plt.show()
